In [18]:
texts = {'text1' :'I have a dream that my four children will one day live in a nation where they will not be judged by the color of their skin but by the contenct of their character. i have a dream today',
 'text2':'I have a dream that one day down in Alabama, with its vicious racists, .... one day right there in Alabama little black boys and black girls will be able to join hands with little boys and white girls as sisiters and brothers. I have a dream today',
'text3': 'I have a dream that one day every valley shall be esalted , every hill and mountain shall be made low , the rough places will be made plain, and the crooked places will be made straight, and the glory of the Lord shall be revealed, and all fleah shall see it together',
'text4' : 'This is my hope ... With this faith we will be able to hew out of the mountain of despair a stone of hope, With this faith we will be able to transform the jangling discords of our nation into a symphony of brotherhood. With this faith we will be able to work together, to pray together, to struggle together, to go to jail together, to stand up for freedom together, knowing that we will be free one day'}

In [19]:
class SearchEngineBase(object):
    def __init__(self):
        pass
    
    def add_corpus(self, key,text): #读取文件内容
        self.process_corpus(key,text)
        
    def process_corpus(self,key,text):#文件路径为ID，处理后的内容存下来，索引
        raise Exception('search not implemented.')

In [21]:
def main(search_engine):
    for key,text in texts.items() :
        search_engine.add_corpus(key,text)
    A = 3   
    while A:
        query = input()
        results = search_engine.search(query)
        print('found {} result(s): '.format(len(results)))
        for result in results:
            print(result)
        A= A-1

In [26]:
class SimpleEnigine(SearchEngineBase):
    def __init__(self):
        super(SimpleEnigine,self).__init__()
        self.__id_to_texts = {}
    
    def process_corpus(self,key,texts):
        self.__id_to_texts[key] = texts
    
    def search(self,query):
        results = []
        for id,text in self.__id_to_texts.items():
            if query in text:
                results.append(id)
        return results


In [27]:
search_engine = SimpleEnigine()
main(search_engine)

di
found 1 result(s): 
text4
ed
found 3 result(s): 
text1
text3
text4
a
found 4 result(s): 
text1
text2
text3
text4


#### 'BAG OF WORDS'

In [31]:
import re

In [48]:
class BOWEngine(SearchEngineBase):
    def __init__(self):
        super(BOWEngine,self).__init__()
        self.__id_to_words = {}
        
    def process_corpus(self,key,text):
        self.__id_to_words[key] = self.parse_text_to_words(text)
    
    def search(self,query):
        query_words = self.parse_text_to_words(query)
        results = []
        for id,words in self.__id_to_words.items():
            if self.query_match(query_words,words):
                results.append(id)
        return results
    
    @staticmethod
    def query_match(query_words,words):
        for query_word in query_words:
            if query_word not in words:
                return False
        return True
    @staticmethod
    def parse_text_to_words(text):
        text = re.sub(r'[^\w]',' ',text)
        text = text.lower()
        word_list = text.split(' ')
        word_list = filter(None,word_list)
        return set(word_list)
    

In [49]:
search_engine = BOWEngine()
main(search_engine)

i have a dream
found 3 result(s): 
text1
text2
text3
boy and girl
found 0 result(s): 
boy
found 0 result(s): 


####  倒叙索引

In [50]:
class BOWInvertedIndexEngine(SearchEngineBase):
    def __init__(self):
        super(BOWInvertedIndexEngine,self).__init__()
        self.inverted_index ={}
    def process_corpus(self,key,text):
        words = self.parse_text_to_words(text)
        for word in words:
            if word not in self.inverted_index:
                self.inverted_index[word] = []
            self.inverted_index[word].append(id)
            
    def search(self,query):
        query_words = list(self.parse_text_to_words(query))
        query_words_index = list()
        for query_word in query_words:
            query_words_index.append(0)
            
        for query_word in query_words:
            if query_word not in self.inverted_index:
                return []
        
        result = []
        while True:
            current_ids = []
            for idx,query_word in enumerate(query_words):
                current_ids = query_words_index[idx]
                current_inverted_list = self.inverted_index[query_word]
                
                if current_index >= len(current_inverted_list):
                    return result
                current_ids.append(current_inverted_list[current_index])
            if all( x==current_ids[0] for x in current_ids):
                result.append(current_ids[0])
                query_words_index = [x+1 for x in query_words_index]
                continue
            min_val = min(current_ids)
            min_val_pos = current_ids.index(min_val)
            query_words_index[min_val_pos] += 1
        
        @staticmethod
        def parse_text_to_words(text):
            text = re.sub(r'[^\w]',' ',text)
            text = text.lower()
            word_list = text.split(' ')
            word_list = filter(None,word_list)
            return set(word_list)             

####  缓存

In [ ]:
import pylru

In [52]:
class LRUCache(object):
    def __init__(self,size = 32):
        self.cache = pylru.lrucache(size)
    
    def has(self,key):
        return key in self.cache
    
    def get(self,key):
        return self.cache[key]
    
    def set(self,key,value):
        self.cache[key]=value


In [53]:
class BOWInvertedIndexEnigineWithCache(BOWInvertedIndexEngine,LRUCache):
    def __init__(self):
        super(BOWInvertedIndexEnigineWithCache,self).__init__()
        LRUCache.__init__(self)
    
    def search(self,query):
        if self.hae(query):
            print('cache hit!')
            return self.get(query)
        result = super(BOWInvertedIndexEnigineWithCache,self).search(query)
        self.set(query,result)
        return result

#### 多重继承的构造方法

In [ ]:
super(BOWInvertedIndexEnigineWithCache,self).__init__()


#最顶层的父类必须继承object